In [2]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf 
import numpy as np
df = pd.read_csv('data/Default.csv', index_col=0)
df.head() #just checking

,default,student,balance,income
1,No,No,729.526495,44361.625074
2,No,Yes,817.180407,12106.134700
3,No,No,1073.549164,31767.138947
4,No,No,529.250605,35704.493935
5,No,No,785.655883,38463.495879


In [3]:
np.random.seed(0)

question 6 (a)
Answer: Estimated standard errors for the coefficients associated with income and balance are 4.99e-06 and 0 respectively.

In [4]:
#using generalized linear models with statsmodel
#see the wikipedia reference to understand why family is binomial
mod1 = smf.glm(formula='default ~ income + balance', data=df, family=sm.families.Binomial()).fit() #create & fit model
print(mod1.summary()) #show results

                        Generalized Linear Model Regression Results                        
Dep. Variable:     ['default[No]', 'default[Yes]']   No. Observations:                10000
Model:                                         GLM   Df Residuals:                     9997
Model Family:                             Binomial   Df Model:                            2
Link Function:                               logit   Scale:                             1.0
Method:                                       IRLS   Log-Likelihood:                -789.48
Date:                             Tue, 06 Mar 2018   Deviance:                       1579.0
Time:                                     08:49:49   Pearson chi2:                 6.95e+03
No. Iterations:                                  9                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     

question 6 (b)

In [5]:

def boot_fn(default):
    mod1 = smf.glm(formula='default ~ income + balance', data=default, family=sm.families.Binomial()).fit()
    coef_income = mod1.params[1]
    coef_balance = mod1.params[2]
    return [coef_income, coef_balance]
boot_fn(df)

[-2.0808975528986917e-05, -0.0056471029503164915]

question 6 (c)
Answer: Calling boot function n times and applying boot_fn to compute the coefficients average value. n is set to be 100 to have convergent results.

In [12]:
#bootstrap function
def boot(X, bootSample_size=None):
    '''
    Sampling observations from a dataframe
    
    Parameters
    ------------
    X : pandas dataframe
        Data to be resampled
        
    bootSample_size: int, optional
        Dimension of the bootstrapped samples
    
    Returns
    ------------
    bootSample_X : pandas dataframe
        Resampled data
        
    Examples
    ----------
    To resample data from the X dataframe:
        >> boot(X)
    The resampled data will have length equal to len(X).
    
    To resample data from the X dataframe in order to have length 5:
        >> boot(X,5)
    
    References
    ------------
    http://nbviewer.jupyter.org/gist/aflaxman/6871948
    
    '''
    
    #assign default size if non-specified
    if bootSample_size == None:
        bootSample_size = len(X)
    
    #create random integers to use as indices for bootstrap sample based on original data
    bootSample_i = (np.random.rand(bootSample_size)*len(X)).astype(int)
    bootSample_i = np.array(bootSample_i)
    bootSample_X = X.iloc[bootSample_i]
    
    return bootSample_X

In [13]:
#running model for bootstrapped samples
coefficients = [] #variable initialization
n = 100 #number of bootstrapped samples

for i in range(0,n):
    coef_i = boot_fn(boot(df)) #determining coefficients for specific bootstrapped sample
    coefficients.append(coef_i) #saving coefficients value

print(pd.DataFrame(coefficients).mean()) #print average of coefficients

0   -0.000020
1   -0.005655
dtype: float64


question 6 (d)
Answer: Similar answers obtained corrected to the second and third significant digits.